In [1]:
# original R code by Ty Tuff, translated into Python

!conda activate SCE

In [2]:
# imports
import os
from osgeo import gdal
import rasterio
import datacube
import requests
import json
from datetime import datetime
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Polygon
from dateutil.relativedelta import relativedelta
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
from rasterio.plot import show
from osgeo import osr
import tmap
from datacube import Datacube
from datacube.utils.geometry import Geometry
from matplotlib.animation import FuncAnimation

In [3]:
# access the API and download scenes
# Define the API Hub search endpoint for Sentinel-2
search_url = 'https://apihub.copernicus.eu/apihub/search'

# Copernicus Open Access Hub credentials
username = 'your_username'
password = 'your_password'

# Bounding box coordinates (longitude and latitude) WIP: Replace with SCE

bbox = {
    "bottom": 52.0,
    "top": 53.0,
    "left": 8.0,
    "right": 9.0
}

# Search parameters
search_params = {
    'q': f'footprint:"Intersects({bbox["top"]}, {bbox["left"]}, {bbox["bottom"]}, {bbox["right"]})" producttype:S2MSI1C',
    'rows': 5,  # Number of search results to retrieve (their API only allows 2 at a time?)
    'start': 0,
}

# Authenticate with the API
auth_response = requests.post('https://apihub.copernicus.eu/apihub/tokens', data={
    'username': username,
    'password': password,
})
auth_response.raise_for_status()
token = auth_response.json()['token']

# Search for Sentinel-2 products
search_response = requests.get(search_url, params=search_params, headers={'Authorization': f'Bearer {token}'})
search_response.raise_for_status()
search_results = search_response.json()

# Loop through search results and download products
download_dir = 'sentinel_data'
os.makedirs(download_dir, exist_ok=True)

for product in search_results.get('products', []):
    # Extract product details
    product_id = product['id']
    product_title = product['title']
    product_url = product['link']

    # Download the product
    product_response = requests.get(product_url, stream=True)
    product_response.raise_for_status()

    # Save the product to a file
    product_filename = os.path.join(download_dir, f'{product_id}_{product_title}.zip')
    with open(product_filename, 'wb') as file:
        for chunk in product_response.iter_content(chunk_size=8192):
            file.write(chunk)

    print(f'Downloaded: {product_title}')

print('Download complete.')

HTTPError: 404 Client Error:  for url: https://apihub.copernicus.eu/apihub/tokens

In [ ]:
# data cube (data cubes consist of spatial (x, y) and temporal (time) dimensions, add cloud masking and compositing satellite images).

# Set up the Datacube
dc = Datacube()

# Define the query parameters
query = {
    'time': ('2021-06-01', '2021-07-30'),
    'x': (bbox_32618[0], bbox_32618[2]),
    'y': (bbox_32618[1], bbox_32618[3]),
    'crs': 'EPSG:32618',
    'resolution': (-1, 1),  # Adjust resolution as needed
}

In [ ]:
# bounding box transformation (retrieve bounding box and transform to different coordinate systems (EPSG:4326 and EPSG:32618 in Ty's code).


# Define the bounding box in EPSG:4326
osm_bbox = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
osm_bbox = osm_bbox[osm_bbox['name'] == "United States"]
bbox_4326 = osm_bbox.to_crs("EPSG:4326").total_bounds

# Define the bounding box in EPSG:32618
bbox_32618 = osm_bbox.to_crs("EPSG:32618").total_bounds

In [ ]:
# Sentinel-2 data retrieval (this section interacts with the AWS Earth Search API to search for Sentinel-2 imagery within a specified time range and bounding box. It then fetches the metadata of the retrieved items. Should I stick with this API or use Open Copernicus?) WIP

# Perform the query
dataset = dc.load(product='sentinel_s2_l2a', **query)


In [ ]:
# data Cube Configuration (configure a data cube with specific spatial and temporal parameters, such as resolution, aggregation method, and time extent).




In [ ]:
# NDVI Calculation and Visualization (calculates NDVI, apply a mask to exclude clouds and cloud shadows, 
#compute NDVI values, and visualizes them as a raster plot using various colors for different NDVI values).

# Calculate NDVI
ndvi = (dataset['B08'] - dataset['B04']) / (dataset['B08'] + dataset['B04'])




In [ ]:
# plot median NDVI values

plt.imshow(ndvi.median(dim='time'), cmap='RdYlGn', vmin=-1, vmax=1)
plt.colorbar()
plt.show()

# Load raster data
with rasterio.open('path_to_new_star_raster.tif') as src:
    data = src.read(1)

# Define a color map for the NDVI values
cmap = plt.get_cmap('YlGn')

# Create the NDVI plot
plt.figure(figsize=(8, 8))
plt.imshow(data, cmap=cmap, vmin=-0.2, vmax=1)
plt.colorbar()
plt.title('NDVI')
plt.show()


In [ ]:
# animated NDVI (create animated GIF showing changes in NDVI over time for specified area - save prev result at ani)

# Load the raster data
with rasterio.open('path_to_raster.tif') as src:
    data = src.read(1)

# Define a color map for the NDVI values
cmap = plt.get_cmap('YlGn')

# Create a figure and axis for the initial plot
fig, ax = plt.subplots(figsize=(8, 8))
img = ax.imshow(data, cmap=cmap, vmin=-0.2, vmax=1)
colorbar = plt.colorbar(img)
ax.set_title('NDVI')

# Define an update function to animate the plot
def update(frame):
    # Update the data for each frame (modify this part as needed)
    new_data = data + (frame / 100.0)  # Example: shifting the data
    img.set_data(new_data)
    return img,

# Create the animation
ani = FuncAnimation(fig, update, frames=range(100), interval=100, repeat=False)

# Display the animation
plt.show()